In [305]:
# Import utility functions from utils.ipynb
from utils import normalize_text,write_df_to_snowflake_with_mapping,get_snowflake_connection_from_yaml
import pandas as pd
import os
import re
from IPython.display import display

DATA_PATH = 'Preprocessed Datasets/'


In [306]:
# Load datasets
df_inventory = pd.read_csv(os.path.join(DATA_PATH, 'inventory_dataset.csv'))
df_product = pd.read_csv(os.path.join(DATA_PATH, 'product_dataset.csv'))
df_sales = pd.read_csv(os.path.join(DATA_PATH, 'sales_transaction_dataset.csv'))
df_store = pd.read_csv(os.path.join(DATA_PATH, 'store_dataset.csv'))

print('Inventory DataFrame:')
display(df_inventory)
print(df_inventory.dtypes)

Inventory DataFrame:


,store_id,product_id,stock_quantity,last_updated
0,STORE20,PROD136,85.0,2025-09-14 15:00:09
1,STORE20,PROD177,98.0,2025-09-26 03:35:46
2,STORE83,PROD31,49.0,2025-09-24 10:43:39
3,STORE78,PROD192,67.0,2025-09-09 05:33:38
4,STORE75,PROD44,87.0,2025-09-27 01:52:08
...,...,...,...,...
4995,STORE26,PROD70,12.0,2025-09-24 05:41:17
4996,STORE86,PROD51,54.0,2025-09-29 02:03:33
4997,STORE79,PROD157,161.0,2025-09-07 11:20:59
4998,STORE38,PROD177,146.0,2025-09-06 21:42:57


store_id           object
product_id         object
stock_quantity    float64
last_updated       object
dtype: object


In [307]:
# Remove duplicates
df_inventory_dis = df_inventory.drop_duplicates()
df_product_dis = df_product.drop_duplicates()
df_sales_dis = df_sales.drop_duplicates()
df_store_dis = df_store.drop_duplicates()

print('Inventory DataFrame:')
display(df_inventory_dis)
print(df_inventory_dis.dtypes)

Inventory DataFrame:


,store_id,product_id,stock_quantity,last_updated
0,STORE20,PROD136,85.0,2025-09-14 15:00:09
1,STORE20,PROD177,98.0,2025-09-26 03:35:46
2,STORE83,PROD31,49.0,2025-09-24 10:43:39
3,STORE78,PROD192,67.0,2025-09-09 05:33:38
4,STORE75,PROD44,87.0,2025-09-27 01:52:08
...,...,...,...,...
4995,STORE26,PROD70,12.0,2025-09-24 05:41:17
4996,STORE86,PROD51,54.0,2025-09-29 02:03:33
4997,STORE79,PROD157,161.0,2025-09-07 11:20:59
4998,STORE38,PROD177,146.0,2025-09-06 21:42:57


store_id           object
product_id         object
stock_quantity    float64
last_updated       object
dtype: object


In [308]:
def pad_id(id_value, padlen):

    m = re.match(r'^([A-Z]+)(\d+)$', str(id_value).strip())
    if m:
        return f'{m.group(1)}{m.group(2).zfill(padlen)}'
    return id_value


In [309]:
# Normalize text fields
df_product_dis['product_name'] = df_product_dis['product_name'].apply(normalize_text)
df_store_dis['store_name'] = df_store_dis['store_name'].apply(normalize_text)
df_inventory_dis['store_id'] = df_inventory_dis['store_id'].apply(lambda x: pad_id(x, 3))
df_inventory_dis['product_id'] = df_inventory_dis['product_id'].apply(lambda x: pad_id(x, 5))
df_sales_dis['store_id'] = df_sales_dis['store_id'].apply(lambda x: pad_id(x, 3))
df_sales_dis['product_id'] = df_sales_dis['product_id'].apply(lambda x: pad_id(x, 5))


print("Products (head):")
display(df_inventory_dis)
print("Stores (head):")
print(df_store_dis.head(), '\n')

print("Inventory (head):")
print(df_inventory_dis.head(), '\n')

print("Sales (head):")
print(df_sales_dis.head(), '\n')


Products (head):


,store_id,product_id,stock_quantity,last_updated
0,STORE020,PROD00136,85.0,2025-09-14 15:00:09
1,STORE020,PROD00177,98.0,2025-09-26 03:35:46
2,STORE083,PROD00031,49.0,2025-09-24 10:43:39
3,STORE078,PROD00192,67.0,2025-09-09 05:33:38
4,STORE075,PROD00044,87.0,2025-09-27 01:52:08
...,...,...,...,...
4995,STORE026,PROD00070,12.0,2025-09-24 05:41:17
4996,STORE086,PROD00051,54.0,2025-09-29 02:03:33
4997,STORE079,PROD00157,161.0,2025-09-07 11:20:59
4998,STORE038,PROD00177,146.0,2025-09-06 21:42:57


Stores (head):
   store_id                  store_name   region             city
0  STORE001     Reichert Mayer And Mann     East   Port Sarailand
1  STORE002         Wintheiser And Sons     East  Port Jailynfort
2  STORE003                 Ebert Group     West            Davis
3  STORE004             Marks - Dickens    South        Eulahstad
4  STORE005  Monahan Hammes And Reinger  Central        Harberton 

Inventory (head):
   store_id product_id  stock_quantity         last_updated
0  STORE020  PROD00136            85.0  2025-09-14 15:00:09
1  STORE020  PROD00177            98.0  2025-09-26 03:35:46
2  STORE083  PROD00031            49.0  2025-09-24 10:43:39
3  STORE078  PROD00192            67.0  2025-09-09 05:33:38
4  STORE075  PROD00044            87.0  2025-09-27 01:52:08 

Sales (head):
  transaction_id  store_id product_id  quantity_sold            sale_date  \
0      TXN000001  STORE099  PROD00380             61  2025-09-10 06:43:57   
1      TXN000002  STORE079  PROD00026  


After cleaning and transforming the raw data, save the processed datasets for downstream ETL steps.

In [310]:
# Standardize datetime fields
df_inventory_dis['last_updated'] = pd.to_datetime(df_inventory_dis['last_updated'], errors="coerce")
df_inventory_dis['last_updated'] = df_inventory_dis['last_updated'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_sales_dis['sale_date'] = pd.to_datetime(df_sales_dis['sale_date'], errors='coerce')
df_sales_dis['sale_date'] = df_sales_dis['sale_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
display(df_inventory_dis)
print(df_inventory_dis.dtypes)

,store_id,product_id,stock_quantity,last_updated
0,STORE020,PROD00136,85.0,2025-09-14 15:00:09
1,STORE020,PROD00177,98.0,2025-09-26 03:35:46
2,STORE083,PROD00031,49.0,2025-09-24 10:43:39
3,STORE078,PROD00192,67.0,2025-09-09 05:33:38
4,STORE075,PROD00044,87.0,2025-09-27 01:52:08
...,...,...,...,...
4995,STORE026,PROD00070,12.0,2025-09-24 05:41:17
4996,STORE086,PROD00051,54.0,2025-09-29 02:03:33
4997,STORE079,PROD00157,161.0,2025-09-07 11:20:59
4998,STORE038,PROD00177,146.0,2025-09-06 21:42:57


store_id           object
product_id         object
stock_quantity    float64
last_updated       object
dtype: object


In [311]:
# # Save cleaned datasets
# cleaned_path = 'Preprocessed Datasets/Cleaned/'
# os.makedirs(cleaned_path, exist_ok=True)

# df_inventory_dis.to_csv(os.path.join(cleaned_path, 'inventory_dataset_cleaned.csv'), index=False)
# df_product_dis.to_csv(os.path.join(cleaned_path, 'product_dataset_cleaned.csv'), index=False)
# df_sales_dis.to_csv(os.path.join(cleaned_path, 'sales_transaction_dataset_cleaned.csv'), index=False)
# df_store_dis.to_csv(os.path.join(cleaned_path, 'store_dataset_cleaned.csv'), index=False)
# print('Cleaned datasets saved to:', cleaned_path)

In [ ]:
conn = get_snowflake_connection_from_yaml()
cs = conn.cursor()
# cs.execute("DELETE FROM SNOWFLAKE_LEARNING_DB.PUBLIC.RAW_INVENTORY;")
# cs.execute("DELETE FROM SNOWFLAKE_LEARNING_DB.PUBLIC.RAW_PRODUCT;")
# cs.execute("DELETE FROM SNOWFLAKE_LEARNING_DB.PUBLIC.RAW_SALES_TRANSACTION;")
# cs.execute("DELETE FROM SNOWFLAKE_LEARNING_DB.PUBLIC.RAW_STORE;")

# Write DataFrames to Snowflake tables
write_df_to_snowflake_with_mapping(df_inventory_dis, 'RAW_INVENTORY', conn)
write_df_to_snowflake_with_mapping(df_product_dis, 'RAW_PRODUCT', conn)
write_df_to_snowflake_with_mapping(df_sales_dis, 'RAW_SALES_TRANSACTION', conn)
write_df_to_snowflake_with_mapping(df_store_dis, 'RAW_STORE', conn)
print('DataFrames written to Snowflake tables.')
cs.close()

DataFrames written to Snowflake tables.


True